In [10]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
Meta-learning pipeline with EWC using pre-computed Fisher matrix
- Loads Fisher matrix from fisher_matrix.npz
- Integrates EWC for continual learning
- Includes HVAC-aware features and flowering-period focus
"""

import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models

# =============================
# Hyperparameters
# =============================
SEQ_LEN = 64
FEATURE_DIM = 64
BATCH_SIZE = 32
EPOCHS_META = 20
INNER_LR = 1e-2
META_LR = 1e-3
NUM_CLASSES = 3
NUM_TASKS = 5
SUPPORT_SIZE = 10
QUERY_SIZE = 20
REPLAY_CAPACITY = 1000
REPLAY_WEIGHT = 0.3
LAMBDA_EWC = 1e-3
FLOWERING_WEIGHT = 2.0
FISHER_MATRIX_PATH = "./ewc_assets/fisher_matrix.npz"
MODEL_WEIGHTS_PATH = "./ewc_assets/model_weights.h5"

# Set random seeds
np.random.seed(42)
tf.random.set_seed(42)

# =============================
# 1) Model Architecture
# =============================
def build_lstm_encoder(seq_len, num_feats, feature_dim=FEATURE_DIM):
    inp = layers.Input(shape=(seq_len, num_feats))
    x_cont = layers.Lambda(lambda z: z[:, :, :3])(inp)
    x = layers.LSTM(feature_dim, unroll=True)(x_cont)
    out = layers.Dense(feature_dim, activation="relu")(x)
    return models.Model(inp, out, name="lstm_encoder")

def build_meta_model(encoder, num_classes=NUM_CLASSES):
    inp = layers.Input(shape=(SEQ_LEN, encoder.input_shape[2]))
    z_enc = encoder(inp)

    # HVAC features
    hvac = layers.Lambda(lambda z: z[:, :, 3:7])(inp)
    hvac_mean = layers.Lambda(lambda z: tf.reduce_mean(z, axis=1))(hvac)
    hvac_shift = layers.Lambda(lambda z: z[:, 1:, :])(hvac)
    hvac_prev = layers.Lambda(lambda z: z[:, :-1, :])(hvac)
    hvac_diff = layers.Lambda(lambda t: tf.abs(t[0] - t[1]))([hvac_shift, hvac_prev])
    hvac_toggle_rate = layers.Lambda(lambda z: tf.reduce_mean(z, axis=1))(hvac_diff)

    hvac_feat = layers.Concatenate()([hvac_mean, hvac_toggle_rate])
    hvac_feat = layers.Dense(16, activation="relu")(hvac_feat)

    x = layers.Concatenate()([z_enc, hvac_feat])
    x = layers.Dense(64, activation="relu")(x)
    x = layers.Dense(32, activation="relu")(x)
    out = layers.Dense(NUM_CLASSES, activation="softmax")(x)
    return models.Model(inp, out, name="meta_lstm_classifier")

# =============================
# 2) Load Fisher Matrix and Model Weights
# =============================
def load_ewc_assets(model, fisher_path=FISHER_MATRIX_PATH, weights_path=MODEL_WEIGHTS_PATH):
    """Load pre-computed Fisher matrix and model weights"""
    if not os.path.exists(fisher_path):
        raise FileNotFoundError(f"Fisher matrix file not found: {fisher_path}")
    if not os.path.exists(weights_path):
        raise FileNotFoundError(f"Model weights file not found: {weights_path}")
    
    # Load Fisher matrix
    fisher_data = np.load(fisher_path)
    fisher_matrix = [tf.constant(arr) for arr in fisher_data.values()]
    
    # Load model weights
    model.load_weights(weights_path)
    prev_weights = [tf.identity(w) for w in model.trainable_variables]
    
    print("Successfully loaded EWC assets:")
    print(f"- Fisher matrix shape: {[f.shape for f in fisher_matrix]}")
    print(f"- Model weights shape: {[w.shape for w in prev_weights]}")
    
    return fisher_matrix, prev_weights

# Initialize models
lstm_encoder = build_lstm_encoder(SEQ_LEN, 7)  # Assuming 7 input features
meta_model = build_meta_model(lstm_encoder)

# Load EWC assets
try:
    fisher_matrix, prev_weights = load_ewc_assets(meta_model)
    print("EWC initialization complete")
except Exception as e:
    print(f"Error loading EWC assets: {e}")
    fisher_matrix, prev_weights = None, None

# =============================
# 3) Meta-Learning with EWC
# =============================
class ReplayBuffer:
    def __init__(self, capacity=REPLAY_CAPACITY):
        self.buffer = []
        self.capacity = capacity
        self.n_seen = 0
    
    def add(self, X, y):
        for xi, yi in zip(X, y):
            self.n_seen += 1
            if len(self.buffer) < self.capacity:
                self.buffer.append((xi, yi))
            else:
                r = np.random.randint(0, self.n_seen)
                if r < self.capacity:
                    self.buffer[r] = (xi, yi)
    
    def sample(self, batch_size):
        batch_size = min(batch_size, len(self.buffer))
        idxs = np.random.choice(len(self.buffer), batch_size, replace=False)
        X_s, y_s = zip(*[self.buffer[i] for i in idxs])
        return np.array(X_s), np.array(y_s)
    def size(self):
        return len(self.buffer)
memory = ReplayBuffer()

def sample_tasks(X, y, num_tasks=NUM_TASKS, support_size=SUPPORT_SIZE, query_size=QUERY_SIZE):
    tasks = []
    n = len(X)
    if n < support_size + query_size:
        raise ValueError(f"Not enough samples: need {support_size+query_size}, got {n}")
    for _ in range(num_tasks):
        idx = np.random.choice(n, support_size + query_size, replace=False)
        X_support, y_support = X[idx[:support_size]], y[idx[:support_size]]
        X_query, y_query = X[idx[support_size:]], y[idx[support_size:]]
        tasks.append((X_support, y_support, X_query, y_query))
    return tasks

def inner_update(model, X_support, y_support, lr_inner=INNER_LR):
    with tf.GradientTape() as tape:
        preds_support = model(X_support, training=True)
        loss_support = tf.reduce_mean(
            tf.keras.losses.sparse_categorical_crossentropy(y_support, preds_support)
        )
    grads_inner = tape.gradient(loss_support, model.trainable_variables)
    updated_vars = [w - lr_inner * g for w, g in zip(model.trainable_variables, grads_inner)]
    return updated_vars

def is_flowering_seq(x_seq, light_idx=2, th_light=550.0):
    light_mean = float(np.mean(x_seq[:, light_idx]))
    return light_mean >= th_light

def hvac_toggle_score(x_seq, hvac_slice=slice(3,7), th_toggle=0.15):
    hv = x_seq[:, hvac_slice]
    if hv.shape[0] < 2:
        return 0.0, False
    diff = np.abs(hv[1:] - hv[:-1])
    rate = float(diff.mean())
    return rate, rate >= th_toggle

def train_with_ewc(meta_model, X_train, y_train, fisher_matrix, prev_weights):
    meta_optimizer = tf.keras.optimizers.Adam(META_LR)
    
    for epoch in range(EPOCHS_META):
        tasks = sample_tasks(X_train, y_train)
        epoch_loss, epoch_acc = 0, 0
        
        for X_support, y_support, X_query, y_query in tasks:
            orig_vars = [tf.identity(w) for w in meta_model.trainable_variables]
            
            # Inner update
            updated_vars = inner_update(meta_model, X_support, y_support)
            for var, upd in zip(meta_model.trainable_variables, updated_vars):
                var.assign(upd)
            
            with tf.GradientTape() as tape:
                # Task loss
                preds_q = meta_model(X_query, training=True)
                task_loss = tf.reduce_mean(
                    tf.keras.losses.sparse_categorical_crossentropy(y_query, preds_q)
                )
                total_loss = task_loss
                
                # Replay loss
                if memory.size() >= 8:
                #if len(memory) >= 8:
                    X_old, y_old = memory.sample(32)
                    preds_old = meta_model(X_old, training=True)
                    replay_loss = tf.reduce_mean(
                        tf.keras.losses.sparse_categorical_crossentropy(y_old, preds_old)
                    )
                    total_loss = (1 - REPLAY_WEIGHT) * total_loss + REPLAY_WEIGHT * replay_loss
                
                # EWC loss
                if fisher_matrix is not None and prev_weights is not None:
                    ewc_loss = 0.0
                    for w, pw, f in zip(meta_model.trainable_variables, prev_weights, fisher_matrix):
                        ewc_loss += tf.reduce_sum(f * tf.square(w - pw))
                    total_loss += LAMBDA_EWC * ewc_loss
                
                # Flowering focus
                flowering_mask = []
                toggle_scores = []
                for i in range(len(X_query)):
                    x_seq = X_query[i]
                    flw = is_flowering_seq(x_seq)
                    tscore, tabove = hvac_toggle_score(x_seq)
                    flowering_mask.append(bool(flw and tabove))
                    toggle_scores.append(tscore)
                
                if any(flowering_mask):
                    ratio = sum(flowering_mask) / len(flowering_mask)
                    mean_toggle = np.mean([t for m,t in zip(flowering_mask, toggle_scores) if m]) if any(flowering_mask) else 0.0
                    toggle_boost = min(1.0 + float(mean_toggle)*2.0, FLOWERING_WEIGHT)
                    boost = 1.0 + (FLOWERING_WEIGHT - 1.0) * ratio
                    total_boost = float(min(boost * toggle_boost, FLOWERING_WEIGHT))
            
            grads = tape.gradient(total_loss, meta_model.trainable_variables)
            
            if any(flowering_mask):
                grads = [g * total_boost for g in grads]
            
            meta_optimizer.apply_gradients(zip(grads, meta_model.trainable_variables))
            
            # Metrics
            epoch_loss += float(task_loss.numpy())
            epoch_acc += float(
                tf.reduce_mean(
                    tf.cast(
                        tf.equal(tf.argmax(preds_q, axis=1), y_query),
                        tf.float32
                    )
                )
            ) 
            
            # Restore original vars
            for var, orig in zip(meta_model.trainable_variables, orig_vars):
                var.assign(orig)
            
            # Update memory
            memory.add(X_support, y_support)
            memory.add(X_query, y_query)
        
        avg_loss = epoch_loss / len(tasks)
        avg_acc = epoch_acc / len(tasks)
        print(f"Epoch {epoch+1}/{EPOCHS_META} - Loss: {avg_loss:.4f} - Acc: {avg_acc:.4f}")

# =============================
# 4) Example Usage
# =============================
if __name__ == "__main__":
    # Generate dummy data if no real data available
    # In practice, replace with your actual data loading code
    print("Generating dummy data for demonstration...")
    X_dummy = np.random.randn(100, SEQ_LEN, 7).astype(np.float32)
    y_dummy = np.random.randint(0, NUM_CLASSES, size=100).astype(np.int32)
    
    if fisher_matrix is not None and prev_weights is not None:
        print("\nStarting training with EWC...")
        train_with_ewc(meta_model, X_dummy, y_dummy, fisher_matrix, prev_weights)
    else:
        print("\nCould not start training - EWC assets not loaded properly")
    
    # Save updated model
    print("\nSaving updated model...")
    meta_model.save_weights("updated_model_weights.h5")
    print("Training complete and model saved")

Successfully loaded EWC assets:
- Fisher matrix shape: [TensorShape([3, 256]), TensorShape([64, 256]), TensorShape([256]), TensorShape([64, 64]), TensorShape([64]), TensorShape([8, 16]), TensorShape([16]), TensorShape([80, 64]), TensorShape([64]), TensorShape([64, 32]), TensorShape([32]), TensorShape([32, 3]), TensorShape([3])]
- Model weights shape: [TensorShape([3, 256]), TensorShape([64, 256]), TensorShape([256]), TensorShape([64, 64]), TensorShape([64]), TensorShape([8, 16]), TensorShape([16]), TensorShape([80, 64]), TensorShape([64]), TensorShape([64, 32]), TensorShape([32]), TensorShape([32, 3]), TensorShape([3])]
EWC initialization complete
Generating dummy data for demonstration...

Starting training with EWC...
Epoch 1/20 - Loss: 1.1633 - Acc: 0.4300
Epoch 2/20 - Loss: 1.1922 - Acc: 0.3500
Epoch 3/20 - Loss: 1.2252 - Acc: 0.3200
Epoch 4/20 - Loss: 1.2462 - Acc: 0.2700
Epoch 5/20 - Loss: 1.1809 - Acc: 0.3600
Epoch 6/20 - Loss: 1.1540 - Acc: 0.4200
Epoch 7/20 - Loss: 1.1598 - Ac